# Demographics

In [ ]:
import json
from pathlib import Path

import biopsykit as bp
import matplotlib.pyplot as plt
import seaborn as sns
from fau_colors import cmaps, register_fausans_font

from stresspose_analysis.datasets.mainstudy import MainStudyDataset

%load_ext autoreload
%autoreload 2
%matplotlib widget

In [ ]:
register_fausans_font()
plt.close("all")

palette = sns.color_palette(cmaps.faculties)
sns.set_theme(context="notebook", style="ticks", palette=palette)

plt.rcParams["figure.figsize"] = (8, 4)
plt.rcParams["pdf.fonttype"] = 42
plt.rcParams["mathtext.default"] = "regular"
plt.rcParams["font.family"] = "sans-serif"
plt.rcParams["font.sans-serif"] = "FAUSans Office"


palette

## Setup Paths

In [ ]:
deploy_type = "local"

In [ ]:
config_dict = json.load(Path("../../config.json").open(encoding="utf-8"))
base_path = Path(config_dict[deploy_type]["base_path"])
base_path

In [ ]:
output_path = Path("../../results")
bp.utils.file_handling.mkdirs(output_path)

## With Excluded Participants

In [ ]:
dataset = MainStudyDataset(base_path, exclude_without_mocap=False)
dataset

### Load Data

In [ ]:
quest_data = dataset.questionnaire
quest_data_code = bp.utils.dataframe_handling.apply_codebook(quest_data, dataset.codebook)
quest_data_code.head()

In [ ]:
demographics_dict = {}

### Gender

In [ ]:
gender = bp.metadata.gender_counts(quest_data_code, gender_col="Gender")
demographics_dict["gender"] = gender

gender

### Age

In [ ]:
age = quest_data[["Age"]].agg(["mean", "std"])
demographics_dict["age"] = age

age

In [ ]:
bmi = bp.metadata.bmi(quest_data, columns=["Weight", "Height"])
bmi = bmi.agg(["mean", "std"])
demographics_dict["bmi"] = bmi

bmi

## Without Excluded Participants

In [ ]:
dataset = MainStudyDataset(base_path, exclude_without_mocap=True)
dataset

### Load Data

In [ ]:
quest_data = dataset.questionnaire
quest_data_code = bp.utils.dataframe_handling.apply_codebook(quest_data, dataset.codebook)
quest_data_code.head()

### Gender

In [ ]:
gender = bp.metadata.gender_counts(quest_data_code, gender_col="Gender")
demographics_dict["gender_excluded"] = gender

gender

### Age

In [ ]:
age = quest_data[["Age"]].agg(["mean", "std"])
demographics_dict["age_excluded"] = age

age

In [ ]:
bmi = bp.metadata.bmi(quest_data, columns=["Weight", "Height"])
bmi = bmi.agg(["mean", "std"])
demographics_dict["bmi_excluded"] = bmi

bmi

## Export

In [ ]:
bp.io.write_pandas_dict_excel(demographics_dict, output_path.joinpath("demographics.xlsx"))